In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sat May 11 10:40:22 2019

@author: 紅茶
"""

import numpy as np
import math
import matplotlib.pyplot as plt
import tensorflow as tf

ho=2
g=9.8
#Ra = (vo**2) * math.sin(deg*math.pi/180 *2) / g
#Rb = 2 / math.tan(deg*math.pi/180)
#R=Ra+Rb
#H = ( vo*math.sin(deg*math.pi/180) )**2 / (2*g) + 2

# 角度為45度+-10度
# 速度為11.5+-2.5


#data preparation
number_pts = 10000
deg=[]
vo=[]
R=[]
H=[]

for i in range(number_pts):
    deg_tmp = 45 + np.random.normal(0.5, 3.1)
    vo_tmp = 11.5 + np.random.normal(0.1, 0.8)
    deg.append(deg_tmp)
    vo.append(vo_tmp)
    R_tmp = (vo_tmp**2)*math.sin(deg_tmp*math.pi/180*2)/g + 2/math.tan(deg_tmp*math.pi/180)
    H_tmp = ( vo_tmp*math.sin(deg_tmp*math.pi/180) )**2 / (2*g) + 2
    R.append(R_tmp)
    H.append(H_tmp)


#data package
inputs=np.array([vo,deg],dtype=np.float32).transpose()
outcome=np.array([R,H],dtype=np.float32).transpose()

with tf.variable_scope('inputs',reuse=tf.AUTO_REUSE):
    tf_inputs=tf.placeholder(tf.float32,[None,2],name='inputs')
with tf.variable_scope('outcome',reuse=tf.AUTO_REUSE):
    tf_outcome=tf.placeholder(tf.float32,[None,2],name='outcome')


#NN
with tf.variable_scope('Net',reuse=tf.AUTO_REUSE):
    L1=tf.layers.dense(tf_inputs,16,tf.nn.relu,name='hidden_layer1')
    L1_BN=tf.layers.batch_normalization(inputs=L1,training=True,name='batch_normalization_layer1')

    L2=tf.layers.dense(L1_BN,64,tf.nn.relu,name='hidden_layer2')
    L2_BN=tf.layers.batch_normalization(inputs=L2,training=True,name='batch_normalization_layer2')

    L3=tf.layers.dense(L2_BN,16,tf.nn.relu,name='hidden_layer3')
    L3_BN=tf.layers.batch_normalization(inputs=L3,training=True,name='batch_normalization_layer3')

    output=tf.layers.dense(L3_BN,2)



#construct model
logits=output
prediction=tf.nn.softmax(logits)


#loss function,optimizer,initialization,accuracy
loss_op=tf.losses.mean_squared_error(tf_outcome,output,scope='loss')

with tf.variable_scope('train_op',reuse=tf.AUTO_REUSE):
    train_op=tf.train.AdamOptimizer(0.005).minimize(loss_op)

init=tf.global_variables_initializer()


#evaluate model
correct_pred=tf.equal(tf.argmax(prediction,1),tf.argmax(tf_outcome,1))
accuracy=tf.reduce_mean(tf.cast(correct_pred,tf.float32))

#session run
with tf.Session() as sess:
    sess.run(init)
    for step in range(10000):
        _,L,predic=sess.run([train_op,loss_op,output],feed_dict={tf_inputs:inputs,tf_outcome:outcome})
        if step%100==0:
            loss,acc=sess.run([loss_op,accuracy],feed_dict={tf_inputs:inputs,tf_outcome:outcome})
            
            print("Step " + str(step) + ", Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))
            
    print('Optimization Finished!')
            
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={tf_inputs:inputs,tf_outcome:outcome}))

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Step 0, Loss= 139.154831, Training Accuracy= 0.464
Step 100, Loss= 24.829165, Training Accuracy= 0.990
Step 200, Loss= 0.002896, Training Accuracy= 1.000
Step 300, Loss= 0.001042, Training Accuracy= 1.000
Step 400, Loss= 0.000903, Training Accuracy= 1.000
Step 500, Loss= 0.000765, Training Accuracy= 1.000
Step 600, Loss= 0.000593, Training Accuracy= 1.000
Step 700, Loss= 0.001795, Training Accuracy= 1.000
Step 800, Loss= 0.000786, Training Accuracy= 1.000
Step 900, Loss= 0.000479, Training Accuracy= 1.000
Step 1000, Loss= 0.000472, Training Accuracy= 1.000
Step 1100, Loss= 0.000358, Training Accuracy= 1.000
Step 1200, Loss= 0.000346, Training Accuracy= 1.000
Step 1300, Loss= 0.000925, Traini